In [1]:
!ls ./pdfs

Allassane.pdf  Gbagbo.pdf  Houphouet.pdf


In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
import os

In [3]:
pdf_directory =  "./pdfs"
loader = DirectoryLoader(
    pdf_directory,
    glob="**/*.pdf",
    loader_cls=PyPDFLoader
)

documents = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500,        # Taille de caractère
    chunk_overlap=0,      # Chevauchement entre chunks
    length_function=len
)

chunks = text_splitter.split_documents(documents)

In [5]:
# afficher un chunk
chunks[0]

Document(metadata={'producer': 'Skia/PDF m144 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Houphouet', 'source': 'pdfs/Houphouet.pdf', 'total_pages': 27, 'page': 0, 'page_label': '1'}, page_content="Félix  Houphouët-Boigny  est  un  homme  politique français puis  homme  d'État ivoirien,  né  Dia  Houphouët  le  18  octobre  1905  à  N'Gokro (village  ayant  précédé  Yamoussoukro)  selon  la  biographie  officielle  et  mort  le  7  décembre  1993  à  Yamoussoukro.  \nChef  traditionnel,  médecin,  planteur  et  dirigeant  syndical,  il  est  notamment  député  français (1945-1959),  membre  de  gouvernements  français (1956-1961),  président  de  l'Assemblée  nationale  ivoirienne (1953-1959),  maire  d'Abidjan (1956-1960),  Premier  ministre  ivoirien (1959-1960)  et  premier  président  de  la  république  de  Côte  d'Ivoire (1960-1993).  «  Père  »  de  l'indépendance de  son  pays,  fréquemment  surnommé  «  Le  Sage  »  ou  «  Le  Vieux  »,  il  domine

In [6]:
# nom
len(chunks)

109

**Avec 'Embeddinggemma' comme modèle d'Embedding**

In [7]:
embeddings_model =  "embeddinggemma"
embeddings = OllamaEmbeddings(model=embeddings_model)

/tmp/ipykernel_46715/2053745318.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=embeddings_model)


In [9]:
vector_store = FAISS.from_documents(chunks, embeddings)

In [10]:
# charger vector_store dans FAISS
save_path =  "faiss_index"
vector_store.save_local(save_path)

In [11]:
load_path =  save_path
embeddings = OllamaEmbeddings(model=embeddings_model)
vector_store = FAISS.load_local(load_path, embeddings, allow_dangerous_deserialization=True)

In [12]:
query = "Qui sont Allassane, Gbagbo et Houphouet ? "
print(f"\n🔍 Question : {query}")


🔍 Question : Qui sont Allassane, Gbagbo et Houphouet ? 


In [15]:
k = 10
# -------- RETRIEVAL : Recherche des documents pertinents --------
relevant_docs = vector_store.similarity_search(query, k=k)

In [16]:
relevant_docs

[Document(id='c8e3feb5-f987-4171-8bf1-ebf21f625cfc', metadata={'producer': 'Skia/PDF m144 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Gbagbo', 'source': 'pdfs/Gbagbo.pdf', 'total_pages': 12, 'page': 1, 'page_label': '2'}, page_content="(grade  qui  pouvait  être  attribué  à  un  étudiant  n'ayant  pas  suivi  les  cours  de  DEA  \nindispensables\n \npour\n \nsoutenir\n \nune\n \nthèse\n \nde\n \n«\n \ndocteur\n \nd'État\n \n»)\n \nen\n \nhistoire\n \nintitulée\n Les  Ressorts  socio-économiques  de  la  politique  ivoirienne  :  1940-1960  à  l'université  Paris-Diderot\n[\n7\n]\n.  \nAscension  politique  \nOpposant  à  Houphouët-Boigny  \nMembre  du  Syndicat  national  de  la  recherche  et  de  l'enseignement  supérieur  (SYNARES),  ce  syndicaliste actif  dans  les  années  1970 est  emprisonné  à  Séguéla et  Bouaké de  mars  1971 à  janvier  1973.  Devenu  directeur  de  l'IHAAA  en  1980,  il  se  révèle  lors  des  manifestations  étudiantes  du 

In [17]:
context = "\n\n".join([doc.page_content for doc in relevant_docs])

In [18]:
llm_model = "mistral:latest"
# -------- GENERATION : Génération de la réponse --------
llm = Ollama(model=llm_model)

/tmp/ipykernel_46715/823457461.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(model=llm_model)


### Avec Ollama

In [19]:
prompt = f"""
Tu es un assistant educatif, professeur, etc...
Utilise UNIQUEMENT les informations suivantes pour répondre.
Si ce n'est pas dans le contexte, dis que ce n'est pas disponible.

CONTEXTE:
{context}

QUESTION:
{query}

Réponse :
"""

answer = llm.invoke(prompt)

In [20]:
answer

" Alassane Ouattara est un homme politique ivoirien qui a servi comme président de la Côte d'Ivoire de 2011 à 2020. Laurant Gbagbo est également un homme politique ivoirien qui a été président de la Côte d'Ivoire de 2000 à 2011, avant d'être remplacé par Alassane Ouattara. Félix Houphouët-Boigny est un homme politique ivoirien qui fut le premier président de la Côte d'Ivoire après son indépendance en 1960 jusqu'à sa mort en 1993. Il gouverna l'Afrique occidentale française (AOF) avant que celle-ci ne devienne la Côte d'Ivoire."

### Avec Azure DeepSeek

In [46]:
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

In [ ]:
# Ton endpoint et ton modèle
endpoint = "https://models.github.ai/inference"
model = "deepseek/DeepSeek-V3-0324"

# Ton token GitHub
token = "**"

In [48]:
prompt = f"""
Tu es un assistant éducatif.
Utilise UNIQUEMENT les informations suivantes pour répondre.
Si ce n'est pas dans le contexte, dis que ce n'est pas disponible.

CONTEXTE :
{context}

QUESTION :
{query}

Réponse :
"""

In [49]:
client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

# On envoie TON prompt comme UserMessage
response = client.complete(
    messages=[
        SystemMessage("Tu es un assistant RAG spécialisé en éducation."),
        UserMessage(prompt),
    ],
    temperature=0.8,
    top_p=0.1,
    max_tokens=2048,
    model=model
)

KeyboardInterrupt: 

In [78]:
response.choices[0].message.content

'D\'après le contexte fourni, le prénom "Dia" donné à Houphouët signifie dans sa langue "prophète" ou "magicien". Le nom "Houphouët" provient quant à lui du baoulé "ufuɛ" et il s\'agit d\'un nom expiatoire donné aux enfants nés dans des circonstances particulières (par exemple, aux abords d\'un village ou après plusieurs décès successifs d\'enfants dans la famille). \n\nAinsi, "Dia Houphouët" peut être interprété comme "le prophète/magicien Houphouët". Plus tard, il a ajouté "Boigny" (signifiant "le bélier") à son nom, devenant ainsi Dia Houphouët-Boigny.'